# Quantum Molecular Dynamics 

“*In fact, the mere act of opening the box will determine the state of the
cat, although in this case there were three determinate states the cat
could be in: these being* **Alive**, **Dead**, *and* **Bloody Furious**.”
― Terry Pratchett, "Lords and Ladies" 

## High-Level Problem Description

TODO

## Mathematical Foundations

### Input Matrix ###

 The input matrix $H$, which represents the state of the quantum system, is built in the following way:

Let $A$, $B$ be two segments with $n$ points. We define $H$ as:

\begin{equation}
H = \begin{pmatrix}
        0^{n \times n} & D(A,B) \\
        D(A,B)^{T} & 0^{n \times n} \\
     \end{pmatrix}
\end{equation}

Such that each $D_{AB}$ cell contains the **squared euclidean distance** between each component of $A$ and $B$, namely,

\begin{equation}
d(a_i,b_i) = \sqrt{(b_x - a_x)^2 + (b_y - a_y)^2 + (b_z - a_z)^2}.
\label{eq:distanceEq}
\end{equation}

**Th**: Input matrix $H$ is **Hermitian**.

**Proof**: Let $H \in \mathbb{C}^{n} \times \mathbb{C}^{n}$, with $n \in \mathbb{N}$. $H$ is **Hermitian** $\iff H = H^{\dagger}$.

$H^{\dagger}$ is the **adjunct** of $H$, i.e., the **transpose complex conjugate** of $H$, meaning that $H_{ij} = \overline{H_{ji}}$. 

For $0$ cells, complex conjugate is 0. For cells of $D_{AB}$, we know that $d$ is defined as an application $\mathbb{V} \times \mathbb{V} \mapsto \mathbb{R}$. Since for each $x \in \mathbb{R}, \overline{x} = x$, we have that $H = \overline{H}$.

Now, what's left to prove is that $H = H^{T}$. To prove it, we calculate the transpose of $H$ as follows:

$H^{T} = \begin{pmatrix}
           0 & \dots & 0 & D(A,B)_{0,0} & \dots & D(A,B)_{0,n} \\
           \vdots & \ddots & \vdots & \vdots & \ddots & \vdots \\
           0 & \dots & 0 & D(A,B)_{n,0} & \dots & D(A,B)_{n,n}  \\
           D(A,B)_{0,0} & \dots & D(A,B)_{n,0} & 0 & \dots & 0\\
           \vdots & \ddots & \vdots & \vdots & \ddots & \vdots \\
           D(A,B)_{0,n} & \dots & D(A,B)_{n,n} & 0 & \dots & 0\\
        \end{pmatrix}^{T} = \begin{pmatrix}
           0 & \dots & 0 & D(A,B)_{0,0} & \dots & D(A,B)_{0,n} \\
           \vdots & \ddots & \vdots & \vdots & \ddots & \vdots \\
           0 & \dots & 0 & D(A,B)_{n,0} & \dots & D(A,B)_{n,n}  \\
           D(A,B)_{0,0} & \dots & D(A,B)_{n,0} & 0 & \dots & 0\\
           \vdots & \ddots & \vdots & \vdots & \ddots & \vdots \\
           D(A,B)_{0,n} & \dots & D(A,B)_{n,n} & 0 & \dots & 0\\
        \end{pmatrix}$

Since $H_{i,j}^{T} = H_{j,i}.~\mathbf{\square}$

The main advantage of $H$ being Hermitian is that all the eigenvalues are real, therefore are suitable representations for the Hamiltonians of quantum systems.

### Eigenvalues Calculation 

The goal of this work is to calculate distance of two alpha carbon atoms in the same segment or among two different segments. To this end, we need to calculate the lowest eigenvalue of $H$. 

Calculation of eigenvalues in quantum machine is defined as follows: given a Hermitian matrix 𝐻
and a minimum eigenvalue $\lambda_{min}$, associated to a quantum eigenstate $|\psi_{min}\rangle$, we want to find a $\lambda_{\theta}$ such that:

\begin{equation}
\lambda_{min} \leq \lambda_{\theta} \equiv \langle \psi(\theta)|H|\psi(\theta)\rangle
\end{equation}

Where $\psi(\theta)$ is the eigenstate associated to $\lambda_{\theta}$ and $\theta$ is the parameter of the quantum Ansatz, which is used to minimize $\lambda_{\theta}$.

In order to find $\lambda_{\theta}$, we define the problem as the **minimization** of the cost function $C(\theta)$, defined as:

\begin{equation}
C(\theta) = \langle \psi(\theta) | H | \psi(\theta) \rangle
\end{equation}

While the minimization of $C$ can be solved by a standard optimizer, i.e., COBYLA, SPSS, we need to define a **parametrized quantum circuit** for the state $\psi(\theta)$. 

### "Naive" Approach

The main idea behind the naive approach is to use the input $H$ matrix to define a **operator**, i.e., a function over a space of physical states onto another space of physical states. In our case, if $\psi$ is the wavefunction of a quantum system, and $H$ represents a *linear transformation* between basis representing different states of the quantum system. Since $H$ is hermitian, we are sure that:
* Eigenvalues are real;
* Basis corresponding to different eigenvalues are orthogonal.
Orthogonality allows a suitable basis set of vectors to represent quantum system's state. If $\psi$ is an eigenstate, we have
\begin{equation}
H\psi(\theta) = \lambda_{\theta}\psi(\theta)
\end{equation}
Our goal is then to find the smallest $\psi(\theta)$ such that $\lambda_{\theta} \geq \lambda_{min}$. 

#### Qiskit implementation


First of all, we initialize our quantum instance as follows:

In [20]:
from qiskit import IBMQ, Aer
from qiskit.aqua import QuantumInstance
from qiskit.providers.aer.noise import NoiseModel
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter

#Loading data about our backend quantum architecture
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')
backend_sim = Aer.get_backend("qasm_simulator")
backend_real = provider.get_backend("ibmq_lima")
coupling_map = backend_real.configuration().coupling_map
# Extracting noise model for error mitigation
noise_model = NoiseModel.from_backend(backend_real)
# Initializing quantum instance with error mitigation based on noise model of backend_real
quantum_instance = QuantumInstance(backend=backend_sim,
                                   shots=20000,
                                   noise_model=noise_model,
                                   coupling_map=coupling_map,
                                   measurement_error_mitigation_cls=CompleteMeasFitter,
                                   cals_matrix_refresh_period=30)


ibmqfactory.load_account:WARNING:2022-02-25 15:05:19,799: Credentials are already in use. The existing account in the session will be replaced.


Now, we need a standard optimizer to run the variational optimization process. We choose COBYLA optimizer, available from qiskit module, and initialize it.

In [21]:
from qiskit.aqua.components.optimizers import COBYLA

optimizer = COBYLA(maxiter=5000, tol=0.0001)

Afterwards, we transform the input $H$ matrix into an operator.

In [22]:
from qiskit.aqua.operators import MatrixOperator, op_converter
import numpy as np

#Initialization of a 2x2 matrix in the same shape as MD program
dimensions = 2
rdist = np.random.rand(dimensions,dimensions)
rdist[0][0] = 1.0
rdist[0][1] = 2.0
rdist[1][0] = 1.0
rdist[1][1] = 3.0
rdistt = np.transpose(rdist)
z1 = np.zeros((dimensions,dimensions))
z2 = np.zeros((dimensions,dimensions))
in_matrix = np.block([[z1,rdist],[rdistt,z2]])

#Converting matrix into operator
hamiltonian = MatrixOperator(in_matrix)
hamiltonian_qubit_op = op_converter.to_weighted_pauli_operator(hamiltonian)

Now, we need to define our own Ansatz, where the variational algorithm and the operator will be applied. To this end, we use the *RealAmplitudes* wave function, i.e., a heuristic trial wave function used as Ansatz in chemistry
applications or classification circuits in machine learning. The circuit consists of alternating layers of $RY$ and $CX$ entanglements. Another characteristic of this Ansatz is that the amplitudes are always real. We use an Ansatz with $2$ qubits and $2$ repetitions and visualize it.

In [23]:
from qiskit.circuit.library import RealAmplitudes
best_ansatz_ever = RealAmplitudes(2,reps=2)
best_ansatz_ever.draw(output='text')

┌────────────────────────────────────────────────┐
q_0: ┤0                                               ├
     │  RealAmplitudes(θ[0],θ[1],θ[2],θ[3],θ[4],θ[5]) │
q_1: ┤1                                               ├
     └────────────────────────────────────────────────┘

Finally, we run the VQE algorithm on the input qubit operator.

In [24]:
from qiskit.aqua.algorithms import VQE

vqe = VQE(operator=hamiltonian_qubit_op,var_form=best_ansatz_ever,quantum_instance=quantum_instance, optimizer=optimizer)
vqe_result = np.real(vqe.run(backend_sim)['eigenvalue'])
print(vqe_result)

-3.7626953125


#### Performance evaluation

We evaluate the performance of eigenvalue calculation using the "naive" approach, by comparing the results of classic algorithm with VQE. To this end, we calculate the Normalized Root Mean Square Error (NRMSE) between the "classic" and the VQE value over 100 randomly generated matrices.

In [25]:
import math
import scipy.linalg as lin_alg

#storing values of, respectively, classic and quantum eigenvalues
classic = []
quantum = []

for i in range(1,100):
    #Initializing random matrix
    rdist = np.random.rand(dimensions,dimensions)
    rdistt = np.transpose(rdist)
    z1 = np.zeros((dimensions,dimensions))
    z2 = np.zeros((dimensions,dimensions))
    in_matrix = np.block([[z1,rdist],[rdistt,z2]])
    
    #Conversion of matrix into operator
    hamiltonian = MatrixOperator(in_matrix)
    hamiltonian_qubit_op = op_converter.to_weighted_pauli_operator(hamiltonian)
    
    #Calculation of Quantum eigenvalue
    vqe = VQE(operator=hamiltonian_qubit_op,var_form=best_ansatz_ever,quantum_instance=quantum_instance, optimizer=optimizer)
    vqe_result = np.real(vqe.run(backend_sim)['eigenvalue'])
    
    #Calculation of Classic eigenvalue
    classic_result = lin_alg.eigvalsh(in_matrix)[0]
    
    #Storing eigenvalues of current matrix
    classic.append(classic_result)
    quantum.append(vqe_result)

#Calculating mean square error, root mean square error, root mean square error
MSE = np.square(np.subtract(classic,quantum)).mean()
RMSE = math.sqrt(MSE)
print("RMSE: "+str(RMSE))
format_rmse = "{:.4f}".format((RMSE/(max(classic)-min(classic)))*100.0)
print("NRMSE: "+format_rmse+ "%")

RMSE: 0.05340649243796343
NRMSE: 4.1659%


#### Problems with naive approach

* MatrixOperator is deprecated (future version will require to use standard quantum gates, i.e., Pauli, Toffoli gates);
* Complexity of transformation is hidden;
* No control on output of transformation;
* Still different from classic value;
* Does not work with dimensions which are not power of $2$.